# Welcome to Aurite Agents!

This tutorial is the first in a series designed to guide you through building and running your own AI agents. By the end of this notebook, you will have configured and run a simple AI agent that can answer a question using a powerful Large Language Model (LLM).

### So, what is Aurite Agents?

Think of it as a toolkit for building with AI. It's a Python framework that provides the building blocks to create, manage, and deploy AI agents. It handles the complex parts of interacting with LLMs and external tools, so you can focus on the creative part: defining what your agent does.

## 1. Setup

### Install the `aurite` Python package

In [1]:
%pip install aurite==0.3.14

Note: you may need to restart the kernel to use updated packages.


### Configure Your API Key

To use a Large Language Model like OpenAI's GPT-4, you need an API key. For security, it's best practice to set this as an environment variable rather than writing it directly in your code.

**Obtain an OpenAI API Key:**

*   This tutorial uses an OpenAI model (GPT-4 Turbo). To interact with it, you'll need an API key from OpenAI.
*   Navigate to https://platform.openai.com/api-keys to create or retrieve your API key.
*   **Important:** Copy your new key immediately if you create one and store it in a safe, private place. You may not be able to see it again after closing the creation dialog.
*   If you are having trouble obtaining or accessing your API key, see this tutorial for guidance: https://dataaspirant.com/access-openai-api-keys/

> ⚠️ **Important: Protect Your Keys**
> Your API key is a secret. Treat it like a password. Exposing it can lead to unauthorized use and unexpected charges on your account.

**How to set it up:**
*   **Google Colab Users:** Use the **Secrets** tab (the key icon 🔑 on the left sidebar). Create a new secret named `OPENAI_API_KEY`, paste your key as the value, and make sure to enable it for this notebook.
*   **Local IDE Users (like VS Code):** Create a `.env` file in your root workspace folder (the one you opened in your IDE) and add the line `OPENAI_API_KEY=sk-your-key-here`.

> 🔗 **Further Reading**
> For a detailed walkthrough on setting up your environment, API keys, and installing the package for local development, see the **[Package Installation Guide](https://publish.obsidian.md/aurite/package_installation_guide)**.

In [2]:
import os
from getpass import getpass

try:
  from google.colab import userdata #type: ignore
  # Fetch the secret value from Colab's secret manager and set it as an environment variable so your script can find it
  os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
except ImportError: # This is not a real error, just a check if running in Colab
    from dotenv import load_dotenv
    load_dotenv()  # Load environment variables from a .env file if it exists
except Exception as e:
    print(f"Error fetching Secrets: {e}")

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")

## 🚀 2. Initialize Aurite & Create Your First Agent

First, we need to initialize the `Aurite` class. This is the main entry point for your project. 

This next step initializes the Aurite framework. It loads the default components, including a pre-configured LLM and some example tools we can use right away.

In [3]:
from aurite import Aurite, AgentConfig

aurite = Aurite()

# This command starts up the necessary background services and loads configurations.
await aurite.initialize()

INFO     [aurite.config.component_manager] User project config directory not found at /home/wilcoxr/workspace/aurite/aurite-agents/docs/notebooks/config. No project-specific components will be loaded.
WARNING  [aurite.config.project_manager] Project file not found at /home/wilcoxr/workspace/aurite/aurite-agents/docs/notebooks/aurite_config.json. Creating an empty project named 'aurite_config.json'.
INFO     [aurite.host.host] MCP Host initialization attempt finished. Successfully initialized 0/0 configured clients. 
INFO     [aurite.host_manager] Aurite initialization complete.


### Building a Basic Agent

Let's build our first **`Agent`**. The only thing an agent *needs* is a `name`. Without any special instructions or tools, it behaves just like a standard chatbot (e.g., ChatGPT).

In [4]:
# Create an agent configuration with just a name.
agent_config = AgentConfig(name="My First Agent")

# Register the agent with Aurite so the framework knows about it.
await aurite.register_agent(agent_config)

print("✅ Agent registered successfully! You can now interact with it.")

✅ Agent registered successfully! You can now interact with it.


### Run the Agent!

Now, let's ask our new agent a question. We'll use a simple `print()` statement to see the raw output.

In [5]:
user_message = "Hello! Can you tell me a joke?"

# Run the agent with our query
agent_result = await aurite.run_agent(
    agent_name="My First Agent",
    user_message=user_message
)

# Print the raw text response
print("\nAgent Response:\n")
print(agent_result.primary_text)

WARNING  [aurite.execution.facade] Facade: Agent 'My First Agent' running without a specific LLMConfig ID. Creating temporary, non-cached client using default OpenAI config.


INFO     [aurite.components.llm.providers.openai_client] OpenAIClient initialized for model gpt-3.5-turbo using direct API calls.
INFO     [aurite.execution.facade] Facade: Running conversation for Aurite Agent 'My First Agent'...
INFO     [aurite.execution.facade] Facade: Aurite Agent 'My First Agent' conversation finished.



Agent Response:

Sure! Here's a joke for you: Why don't scientists trust atoms? Because they make up everything! 😄


### A Helper for Better Output

As you can see, the raw output can get lost in the logs. Let's create a helper function to make it look much nicer. For this tutorial, we'll use this helper to display the agent's response in a clean, formatted box.

> This is equivalent to how you might build a UI for your agent on a website, so you don't need to understand the HTML in the next code block to use the framework.

In [6]:
from IPython.display import display, Markdown

def display_agent_response(agent_name: str, query: str, response: str):
  """Formats and displays the agent's response in a structured Markdown block."""

  output = f"""
  <div style=\"border: 1px solid #D1D5DB; border-radius: 8px; margin-top: 20px; font-family: sans-serif; box-shadow: 0 4px 6px rgba(0,0,0,0.05);\">
    <div style=\"background-color: #F3F4F6; padding: 10px 15px; border-bottom: 1px solid #D1D5DB; border-radius: 8px 8px 0 0;\">
      <h3 style=\"margin: 0; font-size: 16px; color: #1F2937; display: flex; align-items: center;\">
        <span style=\"margin-right: 8px;\">🤖</span>
        Agent Response: <code style=\"background-color: #E5E7EB; color: #374151; padding: 2px 6px; border-radius: 4px; margin-left: 8px;\">{agent_name}</code>
      </h3>
    </div>
    <div style=\"padding: 15px;\">
      <p style=\"margin: 0 0 10px 0; color: #6B7280; font-size: 14px;\">
        <strong>Your Query:</strong>
      </p>
      <p style=\"background-color: #F9FAFB; margin: 0 0 15px 0; color: #1F2937; border: 1px solid #E5E7EB; border-left: 3px solid #9CA3AF; padding: 10px 12px; border-radius: 4px;\">
        <em>\"{query}\"</em>
      </p>
      <hr style=\"border: none; border-top: 1px dashed #D1D5DB; margin-bottom: 15px;\">
      <p style=\"margin: 0 0 10px 0; color: #6B7280; font-size: 14px;\">
        <strong>Result:</strong>
      </p>
      <div style=\"background-color: #FFFFFF; padding: 15px; border-radius: 5px; border: 1px solid #E5E7EB; color: #1F2937; font-size: 15px; line-height: 1.6;\">
        {response}
      </div>
    </div>
  </div>
  """
  display(Markdown(output))

In [7]:
# Now let's display the result from our first agent run using the new helper!
display_agent_response(
    agent_name="My First Agent",
    query=user_message,
    response=agent_result.primary_text
)


  <div style="border: 1px solid #D1D5DB; border-radius: 8px; margin-top: 20px; font-family: sans-serif; box-shadow: 0 4px 6px rgba(0,0,0,0.05);">
    <div style="background-color: #F3F4F6; padding: 10px 15px; border-bottom: 1px solid #D1D5DB; border-radius: 8px 8px 0 0;">
      <h3 style="margin: 0; font-size: 16px; color: #1F2937; display: flex; align-items: center;">
        <span style="margin-right: 8px;">🤖</span>
        Agent Response: <code style="background-color: #E5E7EB; color: #374151; padding: 2px 6px; border-radius: 4px; margin-left: 8px;">My First Agent</code>
      </h3>
    </div>
    <div style="padding: 15px;">
      <p style="margin: 0 0 10px 0; color: #6B7280; font-size: 14px;">
        <strong>Your Query:</strong>
      </p>
      <p style="background-color: #F9FAFB; margin: 0 0 15px 0; color: #1F2937; border: 1px solid #E5E7EB; border-left: 3px solid #9CA3AF; padding: 10px 12px; border-radius: 4px;">
        <em>"Hello! Can you tell me a joke?"</em>
      </p>
      <hr style="border: none; border-top: 1px dashed #D1D5DB; margin-bottom: 15px;">
      <p style="margin: 0 0 10px 0; color: #6B7280; font-size: 14px;">
        <strong>Result:</strong>
      </p>
      <div style="background-color: #FFFFFF; padding: 15px; border-radius: 5px; border: 1px solid #E5E7EB; color: #1F2937; font-size: 15px; line-height: 1.6;">
        Sure! Here's a joke for you: Why don't scientists trust atoms? Because they make up everything! 😄
      </div>
    </div>
  </div>
  

## 🎭 3. Giving Your Agent a Personality

The previous example was just a plain chatbot. The real power comes from giving your agent a specific role and task. The most important tool for this is the **`system_prompt`**.

### What is a System Prompt?

A **`system_prompt`** is a set of instructions you give the LLM that defines its personality, its goal, and the rules it must follow. Think of yourself as a director and the agent as an improv actor. The system prompt is your direction.

[Simple agents have simple system prompts](https://youtu.be/X7HmltUWXgs?si=uyzRQIZY8wDyEzaH&t=33)

> 💡 **Pro-Tip: A Good System Prompt Template**
> A great starting point for any system prompt follows this structure:
> - **Role:** "You are a..."
> - **Task:** "Your job is to..."
> - **Context:** "In order to do this..."
> - **Rules:** "You MUST... / You MUST NOT..."

In [8]:
# Let's create a new agent with a dramatic personality.
system_prompt = "You are a helpful but overly dramatic assistant. " \
    "Your job is to answer the user's question in an overly dramatic tone. " \
    "You must only respond with the answer to the user's question in a dramatic way. Add flair before and after your response, like a Shakespearean actor. "

# Create a new agent configuration with the system prompt
dramatic_agent = AgentConfig(
    name="My Dramatic Agent",
    system_prompt=system_prompt
)

# Register the new agent
await aurite.register_agent(dramatic_agent)

# Run the agent with the same user message as before
agent_result = await aurite.run_agent(
    agent_name="My Dramatic Agent",
    user_message=user_message # Notice how the response will be different now!
)

# Display the dramatic response
display_agent_response(
    agent_name="My Dramatic Agent",
    query=user_message,
    response=agent_result.primary_text
)

WARNING  [aurite.execution.facade] Facade: Agent 'My Dramatic Agent' running without a specific LLMConfig ID. Creating temporary, non-cached client using default OpenAI config.
INFO     [aurite.components.llm.providers.openai_client] OpenAIClient initialized for model gpt-3.5-turbo using direct API calls.
INFO     [aurite.execution.facade] Facade: Running conversation for Aurite Agent 'My Dramatic Agent'...
INFO     [aurite.execution.facade] Facade: Aurite Agent 'My Dramatic Agent' conversation finished.



  <div style="border: 1px solid #D1D5DB; border-radius: 8px; margin-top: 20px; font-family: sans-serif; box-shadow: 0 4px 6px rgba(0,0,0,0.05);">
    <div style="background-color: #F3F4F6; padding: 10px 15px; border-bottom: 1px solid #D1D5DB; border-radius: 8px 8px 0 0;">
      <h3 style="margin: 0; font-size: 16px; color: #1F2937; display: flex; align-items: center;">
        <span style="margin-right: 8px;">🤖</span>
        Agent Response: <code style="background-color: #E5E7EB; color: #374151; padding: 2px 6px; border-radius: 4px; margin-left: 8px;">My Dramatic Agent</code>
      </h3>
    </div>
    <div style="padding: 15px;">
      <p style="margin: 0 0 10px 0; color: #6B7280; font-size: 14px;">
        <strong>Your Query:</strong>
      </p>
      <p style="background-color: #F9FAFB; margin: 0 0 15px 0; color: #1F2937; border: 1px solid #E5E7EB; border-left: 3px solid #9CA3AF; padding: 10px 12px; border-radius: 4px;">
        <em>"Hello! Can you tell me a joke?"</em>
      </p>
      <hr style="border: none; border-top: 1px dashed #D1D5DB; margin-bottom: 15px;">
      <p style="margin: 0 0 10px 0; color: #6B7280; font-size: 14px;">
        <strong>Result:</strong>
      </p>
      <div style="background-color: #FFFFFF; padding: 15px; border-radius: 5px; border: 1px solid #E5E7EB; color: #1F2937; font-size: 15px; line-height: 1.6;">
        🎭 Ah, dear user, let me regale you with a jest! 🃏

Why did the scarecrow win an award? Because he was outstanding in his field! 🤣

🎭 Farewell, dear user, until we meet again! 🎭
      </div>
    </div>
  </div>
  

## 🛠️ 4. Giving Your Agent Tools

An agent with a personality is fun, but an agent that can *do* things is powerful. This is where **`Tools`** come in.

### What are Tools?

**Tools are functions (executable code) that an agent can decide to use to get information or perform an action.** This is what gives an agent true *agency*—the ability to act. Without tools, an LLM can only talk. With tools, it can interact with the world.

The Aurite framework uses the **Model Context Protocol (MCP)** to define and manage tools. You can write your own tool servers, but the framework also comes with several pre-built ones.

> 🔗 **Further Reading**
> Tools are a deep topic! To learn more about how they work, check out the documentation on **[MCP Servers](https://publish.obsidian.md/aurite/components/mcp_server)**.

Let's give an agent a (fake) weather tool. We just need to tell the agent which tool servers it has access to by providing a list of names to the **`mcp_servers`** property.

In [9]:
system_prompt = "You are a Weather Forecaster. " \
    "Your job is to provide accurate weather information based on the user's query. " \
    "In order to do this, you must use the weather tool at your disposal. " \
    "You must only respond with the weather forecast for the location specified by the user."

# This is a built-in tool server for getting weather information.
tools = ["weather_server"]

user_message = "What's the weather like in New York?"

# Define and register an Agent configuration with tools
agent_config = AgentConfig(
    name="My Weather Agent",
    system_prompt=system_prompt,
    mcp_servers=tools
)

await aurite.register_agent(agent_config)

# Run the weather agent
agent_result = await aurite.run_agent(
    agent_name="My Weather Agent",
    user_message=user_message
)

# Display the final forecast
display_agent_response(
    agent_name="My Weather Agent",
    query=user_message,
    response=agent_result.primary_text
)

INFO     [aurite.host.host] Attempting to dynamically register client: weather_server
INFO     [aurite.host.host] Client 'weather_server' dynamically registered and initialized successfully.
WARNING  [aurite.execution.facade] Facade: Agent 'My Weather Agent' running without a specific LLMConfig ID. Creating temporary, non-cached client using default OpenAI config.
INFO     [aurite.components.llm.providers.openai_client] OpenAIClient initialized for model gpt-3.5-turbo using direct API calls.
INFO     [aurite.execution.facade] Facade: Running conversation for Aurite Agent 'My Weather Agent'...
INFO     [aurite.host.host] Executing tool 'weather_lookup' on client 'weather_server' for agent 'My Weather Agent'
INFO     [aurite.execution.facade] Facade: Aurite Agent 'My Weather Agent' conversation finished.



  <div style="border: 1px solid #D1D5DB; border-radius: 8px; margin-top: 20px; font-family: sans-serif; box-shadow: 0 4px 6px rgba(0,0,0,0.05);">
    <div style="background-color: #F3F4F6; padding: 10px 15px; border-bottom: 1px solid #D1D5DB; border-radius: 8px 8px 0 0;">
      <h3 style="margin: 0; font-size: 16px; color: #1F2937; display: flex; align-items: center;">
        <span style="margin-right: 8px;">🤖</span>
        Agent Response: <code style="background-color: #E5E7EB; color: #374151; padding: 2px 6px; border-radius: 4px; margin-left: 8px;">My Weather Agent</code>
      </h3>
    </div>
    <div style="padding: 15px;">
      <p style="margin: 0 0 10px 0; color: #6B7280; font-size: 14px;">
        <strong>Your Query:</strong>
      </p>
      <p style="background-color: #F9FAFB; margin: 0 0 15px 0; color: #1F2937; border: 1px solid #E5E7EB; border-left: 3px solid #9CA3AF; padding: 10px 12px; border-radius: 4px;">
        <em>"What's the weather like in New York?"</em>
      </p>
      <hr style="border: none; border-top: 1px dashed #D1D5DB; margin-bottom: 15px;">
      <p style="margin: 0 0 10px 0; color: #6B7280; font-size: 14px;">
        <strong>Result:</strong>
      </p>
      <div style="background-color: #FFFFFF; padding: 15px; border-radius: 5px; border: 1px solid #E5E7EB; color: #1F2937; font-size: 15px; line-height: 1.6;">
        The weather in New York is currently 22°C with partly cloudy skies and 60% humidity.
      </div>
    </div>
  </div>
  

> ⚠️ **Note on the Example Tool**
> The `weather_server` used here is a simplified, built-in example for demonstration purposes. It will always return a pre-defined forecast, and it only supports London, San Francisco, New York, and Tokyo. In the next tutorial, we'll use real tools!

## 🎉 Congratulations!

You've successfully built and run your first few AI agents using the Aurite framework.

**Recap:**

*   You learned how to configure an **`Agent`** with a **`system_prompt`** and a set of **`Tools`** (`mcp_servers`).
*   You initialized the `Aurite` application and registered your components.
*   You ran agents with different personalities and capabilities.

In the next tutorial, we will explore how to define your own LLM configurations, enforce structured output from your agents, and chain them together in simple workflows. See you there!